In [1]:
import pandas as pd
import numpy as np

import cehd

In [2]:
pre_assembled_file = 'Input/Processed/osha_data.csv'
raw_database = pd.read_csv(pre_assembled_file, index_col=0)

C:\Users\jmank\AppData\Local\Temp\ipykernel_26036\3671417341.py:2: DtypeWarning: Columns (1,6,10,11,19) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_database = pd.read_csv(pre_assembled_file, index_col=0)


In [3]:
database = cehd.clean_cehd_data(raw_database)

database

,AIR_VOLUME_SAMPLED,BLANK_USED,CITY,DATE_REPORTED,DATE_SAMPLED,EIGHT_HOUR_TWA_CALC,ESTABLISHMENT_NAME,FIELD_NUMBER,IMIS_SUBSTANCE_CODE,INSPECTION_NUMBER,...,SAMPLE_TYPE,SAMPLE_WEIGHT,SAMPLING_NUMBER,SIC_CODE,STATE,SUBSTANCE,TIME_SAMPLED,UNIT_OF_MEASUREMENT,YEAR,ZIP_CODE
0,516.000,N,Plainview,1984-06-15,1984-05-07,NaN,CONTROLLED CASTINGS CORP,2LI8491,T100,111211,...,P,NaN,111211,3365.0,NY,Thorium,258.0,M,1984,11803.0
2,76.000,N,Plainview,1984-06-15,1984-05-07,NaN,CONTROLLED CASTINGS CORP,2LI8493,T100,111211,...,P,NaN,111211,3365.0,NY,Thorium,38.0,M,1984,11803.0
4,74.000,N,East Farmingdale,1984-04-27,1984-04-04,NaN,MID ISLAND NON FERROUS FOUNDRY,2LI84-49,1591,111252,...,P,0.0,111252,3365.0,NY,"Lead, Inorganic (as Pb)",37.0,M,1984,11735.0
7,74.000,N,East Farmingdale,1984-04-27,1984-04-04,NaN,MID ISLAND NON FERROUS FOUNDRY,2LI84-49,1840,111252,...,P,0.0,111252,3365.0,NY,"Nickel, Metal and Insoluble compounds (as Ni)",37.0,M,1984,11735.0
9,74.000,N,East Farmingdale,1984-04-27,1984-04-04,NaN,MID ISLAND NON FERROUS FOUNDRY,2LI84-49,0731,111252,...,P,0.0,111252,3365.0,NY,Copper Fume (as Cu),37.0,M,1984,11735.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395062,64.610,N,nan,2018-03-05,2018-01-23,N,nan,2,2042,806189446,...,P,NaN,806189446,NaN,NaN,p-Phenylene Diamine,65.0,M,2018,00nan
2395063,59.640,N,nan,2018-03-05,2018-01-23,N,nan,3,2042,806189446,...,P,NaN,806189446,NaN,NaN,p-Phenylene Diamine,60.0,M,2018,00nan
2395064,95.424,N,nan,2018-03-05,2018-01-23,N,nan,4,2042,806189446,...,P,NaN,806189446,NaN,NaN,p-Phenylene Diamine,96.0,M,2018,00nan
2395065,61.628,N,nan,2018-03-05,2018-01-23,N,nan,5,2042,806189446,...,P,NaN,806189446,NaN,NaN,p-Phenylene Diamine,62.0,M,2018,00nan


In [9]:
## Exclude substances with few samples
subst = database['IMIS_SUBSTANCE_CODE'].value_counts().reset_index()
subst.columns = ['code', 'n']
where_enough_samples = subst['n'] >= 100
subst = subst[where_enough_samples]

## Remove non-chemical substance codes
non_chemical_codes = ['G301', 'G302', 'Q115', 'T110', 'M125', 'Q116', 'Q100', 'S325']
where_non_chemical = subst['code'].isin(non_chemical_codes)
subst = subst[~where_non_chemical]

sub_list_all = list(subst['code'])
database[database['IMIS_SUBSTANCE_CODE'].isin(sub_list_all)]

,AIR_VOLUME_SAMPLED,BLANK_USED,CITY,DATE_REPORTED,DATE_SAMPLED,EIGHT_HOUR_TWA_CALC,ESTABLISHMENT_NAME,FIELD_NUMBER,IMIS_SUBSTANCE_CODE,INSPECTION_NUMBER,...,SAMPLE_TYPE,SAMPLE_WEIGHT,SAMPLING_NUMBER,SIC_CODE,STATE,SUBSTANCE,TIME_SAMPLED,UNIT_OF_MEASUREMENT,YEAR,ZIP_CODE
4,74.000,N,East Farmingdale,1984-04-27,1984-04-04,NaN,MID ISLAND NON FERROUS FOUNDRY,2LI84-49,1591,111252,...,P,0.000,111252,3365.0,NY,"Lead, Inorganic (as Pb)",37.0,M,1984,11735.0
7,74.000,N,East Farmingdale,1984-04-27,1984-04-04,NaN,MID ISLAND NON FERROUS FOUNDRY,2LI84-49,1840,111252,...,P,0.000,111252,3365.0,NY,"Nickel, Metal and Insoluble compounds (as Ni)",37.0,M,1984,11735.0
9,74.000,N,East Farmingdale,1984-04-27,1984-04-04,NaN,MID ISLAND NON FERROUS FOUNDRY,2LI84-49,0731,111252,...,P,0.000,111252,3365.0,NY,Copper Fume (as Cu),37.0,M,1984,11735.0
10,0.783,N,Passaic,1984-05-18,1984-05-02,NaN,PANTASOTE INC,K168,2580,113951,...,P,NaN,113951,3089.0,NJ,Vinyl Chloride,15.0,P,1984,7055.0
11,0.783,N,Passaic,1984-05-18,1984-05-02,NaN,PANTASOTE INC,K168B,2580,113951,...,P,NaN,113951,3089.0,NJ,Vinyl Chloride,15.0,P,1984,7055.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395030,443.700,N,nan,2018-09-19,2018-09-06,N,nan,03706E,9130,318054095,...,P,0.027,318054095,NaN,NaN,Particulates not otherwise regulated (Respirab...,261.0,M,2018,00nan
2395036,10.830,N,nan,2018-04-10,2018-03-07,N,nan,A1,0560,318091360,...,P,NaN,318091360,NaN,NaN,Carbon Monoxide,361.0,P,2018,00nan
2395037,2.220,N,nan,2018-04-10,2018-03-07,N,nan,A2,0560,318091360,...,P,NaN,318091360,NaN,NaN,Carbon Monoxide,74.0,P,2018,00nan
2395038,17.400,N,nan,2018-04-10,2018-03-07,N,nan,B1,0560,318091360,...,P,NaN,318091360,NaN,NaN,Carbon Monoxide,348.0,P,2018,00nan


In [4]:
r_database = pd.read_parquet('database.parquet')

r_database

,AIR_VOLUME_SAMPLED,BLANK_USED,CITY,DATE_REPORTED,DATE_SAMPLED,EIGHT_HOUR_TWA_CALC,ESTABLISHMENT_NAME,FIELD_NUMBER,IMIS_SUBSTANCE_CODE,INSPECTION_NUMBER,...,SAMPLE_TYPE,SAMPLE_WEIGHT,SAMPLING_NUMBER,SIC_CODE,STATE,SUBSTANCE,TIME_SAMPLED,UNIT_OF_MEASUREMENT,YEAR,ZIP_CODE
0,516.000,N,Plainview,1984-06-15,1984-05-07,NaN,CONTROLLED CASTINGS CORP,2LI8491,T100,111211,...,P,NaN,5245543.0,3365,NY,Thorium,258.0,M,1984,11803
1,76.000,N,Plainview,1984-06-15,1984-05-07,NaN,CONTROLLED CASTINGS CORP,2LI8493,T100,111211,...,P,NaN,5245501.0,3365,NY,Thorium,38.0,M,1984,11803
2,74.000,N,East Farmingdale,1984-04-27,1984-04-04,NaN,MID ISLAND NON FERROUS FOUNDRY,2LI84-49,1591,111252,...,P,0.0,5245345.0,3365,NY,"Lead, Inorganic (as Pb)",37.0,M,1984,11735
3,74.000,N,East Farmingdale,1984-04-27,1984-04-04,NaN,MID ISLAND NON FERROUS FOUNDRY,2LI84-49,1840,111252,...,P,0.0,5245345.0,3365,NY,"Nickel, Metal and Insoluble compounds (as Ni)",37.0,M,1984,11735
4,74.000,N,East Farmingdale,1984-04-27,1984-04-04,NaN,MID ISLAND NON FERROUS FOUNDRY,2LI84-49,0731,111252,...,P,0.0,5245345.0,3365,NY,Copper Fume (as Cu),37.0,M,1984,11735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1506197,64.610,N,nan,2018-03-05,2018-01-23,N,nan,2,2042,806189446,...,P,NaN,WA20180202B,NaN,nan,p-Phenylene Diamine,65.0,M,2018,00NaN
1506198,59.640,N,nan,2018-03-05,2018-01-23,N,nan,3,2042,806189446,...,P,NaN,WA20180202B,NaN,nan,p-Phenylene Diamine,60.0,M,2018,00NaN
1506199,95.424,N,nan,2018-03-05,2018-01-23,N,nan,4,2042,806189446,...,P,NaN,WA20180202B,NaN,nan,p-Phenylene Diamine,96.0,M,2018,00NaN
1506200,61.628,N,nan,2018-03-05,2018-01-23,N,nan,5,2042,806189446,...,P,NaN,WA20180202B,NaN,nan,p-Phenylene Diamine,62.0,M,2018,00NaN
